In [50]:
# 무신사 수집 
import requests
from bs4 import BeautifulSoup as bs
import json
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

import sys
import pandas as pd

import numpy as np
from datetime import date, timedelta
import pandas as pd
from urllib.request import urlopen
import re

import time
import datetime
import os
import warnings

from selenium import webdriver   # 웹 브라우저 자동화
from selenium.webdriver.common.keys import Keys # 브라우저에 입력
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
#import chromedriver_autoinstaller
from selenium.webdriver import ActionChains
#from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

import openpyxl

# 현재 날짜 계산
today = datetime.datetime.now()

In [48]:
pip install openpyxl

     -------------------------------------- 250.0/250.0 KB 7.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Python\eland\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


## comment_data -> json 형식
    url에 상품코드인지 page 조절해서 조회

In [37]:
n = 1
comment_page_list = []

while True:

    url = f'https://mapi.vip.com/vips-mobile/rest/content/reputation/queryBySpuId_for_pc?callback=getCommentDataCb&app_name=shop_pc&app_version=4.0&warehouse=VIP_NH&fdc_area_id=104104101&client=pc&mobile_platform=1&province_id=104104&api_key=70f71280d5d547b2a7bb370a529aeea1&user_id=&mars_cid=1705635889684_99a351c0a065388351254fda836f8dfb&wap_consumer=a&spuId=3258994515834580992&brandId=1711947357&page={n}&pageSize=10&functions=angle&timestamp=1705636089000&keyWordNlp=%E5%85%A8%E9%83%A8-%E6%8C%89%E9%BB%98%E8%AE%A4%E6%8E%92%E5%BA%8F&_=1705635889134'
    headers = {
        'Accept': '*/*',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
        'Cookie': 'mars_cid=1705635889684_99a351c0a065388351254fda836f8dfb; mars_sid=255999d49ac439efb5ec22bc0130f14c; user_class=a; VipUINFO=luc%3Aa%7Csuc%3Aa%7Cbct%3Ac_new%7Chct%3Ac_new%7Cbdts%3A0%7Cbcts%3A0%7Ckfts%3A0%7Cc10%3A0%7Crcabt%3A0%7Cp2%3A0%7Cp3%3A0%7Cp4%3A0%7Cp5%3A1%7Cul%3A3105; vip_first_visitor=1; vip_address=%257B%2522pid%2522%253A%2522104104%2522%252C%2522cid%2522%253A%2522104104101%2522%252C%2522pname%2522%253A%2522%255Cu5e7f%255Cu4e1c%255Cu7701%2522%252C%2522cname%2522%253A%2522%255Cu5e7f%255Cu5dde%255Cu5e02%2522%257D; vip_province=104104; vip_province_name=%E5%B9%BF%E4%B8%9C%E7%9C%81; vip_city_name=%E5%B9%BF%E5%B7%9E%E5%B8%82; vip_city_code=104104101; vip_wh=VIP_NH; vip_ipver=31; vip_access_times=%7B%22detail%22%3A1%7D; pg_session_no=1; vip_tracker_source_from=; VipDFT=6; vpc_uinfo=fr1352%3A0%2Cfr674%3AD1%2Cfr766%3A0%2Cfr896%3A0%2Cfr392%3A310505%2Cfr398%3A0%2Cfr408%3A0%2Cfr251%3AA%2Cfr1195%3A0%2Cfr848%3A0%2Cfr1196%3A0%2Cfr902%3A0%2Cfr901%3A0%2Cfr980%3A0%2Cfr1570%3A0%2Cfr713%3A0%2Cfr1575%3A0%2Cfr1051%3A0%2Cfr259%3AS0-4%2Cfr0901%3A0%2Cfr884%3A0%2Cfr863%3A0%2Cfr1527%3A0%2Cfr344%3A0%2Cfr249%3AA1%2Cfr328%3A3105%2Cfr1544%3A0%2Cfr1543%3A0%2Cfr1521%3A0; mst_area_code=104104; mars_pid=0; visit_id=5CDF2C793EF6D53F382C33A68A572478',
        'Referer': 'https://detail.vip.com/',
        'Sec-Ch-Ua': '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
        'Sec-Ch-Ua-Mobile': '?0',
        'Sec-Ch-Ua-Platform': '"Windows"',
        'Sec-Fetch-Dest': 'script',
        'Sec-Fetch-Mode': 'no-cors',
        'Sec-Fetch-Site': 'same-site',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }

    response = requests.get(url, headers=headers)

    try:
        # HTML을 bs으로 파싱
        soup = bs(response.text, 'html.parser')

        # JavaScript 함수 getCommentDataCb의 호출 부분을 정규 표현식으로 찾기
        pattern = re.compile(r'getCommentDataCb\((.*?)\)', re.DOTALL)
        match = pattern.search(str(soup))

        if match:
            # 괄호 안의 JSON 형식의 데이터 추출
            comment_data_str = match.group(1)

            # JSON 형식의 데이터를 파싱하여 dict로 변환
            comment_data = json.loads(comment_data_str)
            
            if comment_data['data'] == []:
                break
            else:
                comment_page_list.append(comment_data)

        else:
            print('No match found for getCommentDataCb')

    except Exception as e:
        print(f'An unexpected error occurred: {e}')

    n += 1
    
filtered_comment_page_list = [item for item in comment_page_list if isinstance(item, dict) and 'data' in item and item['data']]n

In [16]:
n = 1
comment_page_list = []

while True:

    url = f'https://mapi.vip.com/vips-mobile/rest/content/reputation/queryBySpuId_for_pc?callback=getCommentDataCb&app_name=shop_pc&app_version=4.0&warehouse=VIP_NH&fdc_area_id=104104101&client=pc&mobile_platform=1&province_id=104104&api_key=70f71280d5d547b2a7bb370a529aeea1&user_id=&mars_cid=1705635889684_99a351c0a065388351254fda836f8dfb&wap_consumer=a&spuId=3258994515834580992&brandId=1711947357&page={n}&pageSize=10&functions=angle&timestamp=1705636089000&keyWordNlp=%E5%85%A8%E9%83%A8-%E6%8C%89%E9%BB%98%E8%AE%A4%E6%8E%92%E5%BA%8F&_=1705635889134'
    headers = {
        'Accept': '*/*',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
        'Cookie': 'mars_cid=1705635889684_99a351c0a065388351254fda836f8dfb; mars_sid=255999d49ac439efb5ec22bc0130f14c; user_class=a; VipUINFO=luc%3Aa%7Csuc%3Aa%7Cbct%3Ac_new%7Chct%3Ac_new%7Cbdts%3A0%7Cbcts%3A0%7Ckfts%3A0%7Cc10%3A0%7Crcabt%3A0%7Cp2%3A0%7Cp3%3A0%7Cp4%3A0%7Cp5%3A1%7Cul%3A3105; vip_first_visitor=1; vip_address=%257B%2522pid%2522%253A%2522104104%2522%252C%2522cid%2522%253A%2522104104101%2522%252C%2522pname%2522%253A%2522%255Cu5e7f%255Cu4e1c%255Cu7701%2522%252C%2522cname%2522%253A%2522%255Cu5e7f%255Cu5dde%255Cu5e02%2522%257D; vip_province=104104; vip_province_name=%E5%B9%BF%E4%B8%9C%E7%9C%81; vip_city_name=%E5%B9%BF%E5%B7%9E%E5%B8%82; vip_city_code=104104101; vip_wh=VIP_NH; vip_ipver=31; vip_access_times=%7B%22detail%22%3A1%7D; pg_session_no=1; vip_tracker_source_from=; VipDFT=6; vpc_uinfo=fr1352%3A0%2Cfr674%3AD1%2Cfr766%3A0%2Cfr896%3A0%2Cfr392%3A310505%2Cfr398%3A0%2Cfr408%3A0%2Cfr251%3AA%2Cfr1195%3A0%2Cfr848%3A0%2Cfr1196%3A0%2Cfr902%3A0%2Cfr901%3A0%2Cfr980%3A0%2Cfr1570%3A0%2Cfr713%3A0%2Cfr1575%3A0%2Cfr1051%3A0%2Cfr259%3AS0-4%2Cfr0901%3A0%2Cfr884%3A0%2Cfr863%3A0%2Cfr1527%3A0%2Cfr344%3A0%2Cfr249%3AA1%2Cfr328%3A3105%2Cfr1544%3A0%2Cfr1543%3A0%2Cfr1521%3A0; mst_area_code=104104; mars_pid=0; visit_id=5CDF2C793EF6D53F382C33A68A572478',
        'Referer': 'https://detail.vip.com/',
        'Sec-Ch-Ua': '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
        'Sec-Ch-Ua-Mobile': '?0',
        'Sec-Ch-Ua-Platform': '"Windows"',
        'Sec-Fetch-Dest': 'script',
        'Sec-Fetch-Mode': 'no-cors',
        'Sec-Fetch-Site': 'same-site',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }

    response = requests.get(url, headers=headers)

    try:
        # HTML을 bs으로 파싱
        soup = bs(response.text, 'html.parser')

        # JavaScript 함수 getCommentDataCb의 호출 부분을 정규 표현식으로 찾기
        pattern = re.compile(r'getCommentDataCb\((.*?)\)', re.DOTALL)
        match = pattern.search(str(soup))

        if match:
            # 괄호 안의 JSON 형식의 데이터 추출
            comment_data_str = match.group(1)

            # JSON 형식의 데이터를 파싱하여 dict로 변환
            comment_data = json.loads(comment_data_str)
            
            if comment_data['data'] == []:
                break
            else:
                comment_page_list.append(comment_data)

        else:
            print('No match found for getCommentDataCb')

    except Exception as e:
        print(f'An unexpected error occurred: {e}')

    n += 1
    
filtered_comment_page_list = [item for item in comment_page_list if isinstance(item, dict) and 'data' in item and item['data']]

No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommen

KeyboardInterrupt: 

In [38]:
filtered_comment_page_list[0]['data']

[{'pointVal': 0,
  'reputation': {'address': '浙江',
   'allowShareFlag': '1',
   'cacheIndex': 'cache_01_18_reputation_3258994515834580992_1711947357_全部-按默认排序_1_10_null',
   'content': '',
   'hasVideo': False,
   'imageCount': 1,
   'imageList': [{'imageId': 233321981,
     'url': 'http://h2.appsimg.com/g.vpimg1.com/upload/ugc/p/55/2024/01/11/99/bbf2c40c9f9fc76c48568.6745.jpg',
     'xyPosistion': ''}],
   'imageType': 0,
   'imgRelevance': '1',
   'impresses': '',
   'isAnonymous': 'NO',
   'isCommentConvert': False,
   'isEssence': 'NO',
   'isTop': 'NO',
   'isUseful': False,
   'nlpClass': 'EP',
   'nlpScore': 90,
   'postTime': 1704943698000,
   'repType': '0',
   'reputationId': 609812766,
   'satisfiedStatus': 'VERY_SATISFIED',
   'showDetail': False,
   'sizeInfo': '尺码适中',
   'tagInfos': [{'tagContent': '相当标准',
     'tagId': 2861,
     'tagName': '尺码大小',
     'tagType': '1'},
    {'tagContent': '性价比还可以吧，鞋底软还防滑、去年在得物买了假鞋后悔死了，已经把得物卸载了，还是唯品会最靠谱、正品穿着舒服，现在唯品会就是我的家，什么东西都在唯品会买',
     

In [85]:
# prodName = []
# datelst = []
# sizelst = []
# collst = []
# satisfied= []
# contents = []
# add_date = []
# add_content = []



# for i in range(0, 14):
#     for j in range(0, 9):
#         reputation_data = filtered_comment_page_list[i]['data'][j]

#         #태그형태로 작성
#         if 'tagInfos' in reputation_data['reputation']:
#             prodName.append(reputation_data['reputationProduct']['goodsName'])
#             datelst.append(reputation_data['reputation']['timeDesc'])
#             sizelst.append(reputation_data['reputationProduct']['size'])
#             collst.append(reputation_data['reputationProduct']['colorInfo'])
#             satisfied.append(reputation_data['reputation']['satisfiedStatus'])

#             # Use the same approach to create tag_content_and_name_list for each tagInfos
#             tag_infos = reputation_data['reputation']['tagInfos']
#             tag_content_and_name_list = [f"[{tag_info['tagName']}]{tag_info['tagContent']}" for tag_info in tag_infos]
            
#             # Join the list of strings into a single string
#             tag_content_and_name_str = ', '.join(tag_content_and_name_list)
            
#             contents.append(tag_content_and_name_str)
        
#         #태그를 사용하지 않고 작성
#         else:
#             prodName.append(reputation_data['reputationProduct']['goodsName'])
#             datelst.append(reputation_data['reputation']['timeDesc'])
#             sizelst.append(reputation_data['reputationProduct']['size'])
#             collst.append(reputation_data['reputationProduct']['colorInfo'])
#             satisfied.append(reputation_data['reputation']['satisfiedStatus'])
#             contents.append(reputation_data['reputation']['content'])

In [100]:
prodName = []
datelst = []
sizelst = []
collst = []
satisfied= []
contents = []
add_date = []
add_content = []



for i in range(0, 14):
    for j in range(0, 9):
        reputation_data = filtered_comment_page_list[i]['data'][j]

        #추가 후기를 작성했을 경우
        if 'additionalComments' in reputation_data:
            #태그 형태로 후기를 남긴 경우
            if 'tagInfos' in reputation_data['reputation']:
                prodName.append(reputation_data['reputationProduct']['goodsName'])
                datelst.append(reputation_data['reputation']['timeDesc'])
                sizelst.append(reputation_data['reputationProduct']['size'])
                collst.append(reputation_data['reputationProduct']['colorInfo'])
                satisfied.append(reputation_data['reputation']['satisfiedStatus'])

                # Use the same approach to create tag_content_and_name_list for each tagInfos
                tag_infos = reputation_data['reputation']['tagInfos']
                tag_content_and_name_list = [f"({tag_info['tagName']}){tag_info['tagContent']}" for tag_info in tag_infos]
                
                # Join the list of strings into a single string
                tag_content_and_name_str = ', '.join(tag_content_and_name_list)
                
                contents.append(tag_content_and_name_str)
                add_date.append(reputation_data['additionalComments']['postTimeDesc'])
                add_content.append(reputation_data['additionalComments']['content'])

            else : 
                prodName.append(reputation_data['reputationProduct']['goodsName'])
                datelst.append(reputation_data['reputation']['timeDesc'])
                sizelst.append(reputation_data['reputationProduct']['size'])
                collst.append(reputation_data['reputationProduct']['colorInfo'])
                satisfied.append(reputation_data['reputation']['satisfiedStatus'])
                contents.append(reputation_data['reputation']['content'])
                add_date.append(reputation_data['additionalComments']['postTimeDesc'])
                add_content.append(reputation_data['additionalComments']['content'])



        
        #추가 후기 작성 안함
        else:
            #태그형태로 작성
            if 'tagInfos' in reputation_data['reputation']:
                prodName.append(reputation_data['reputationProduct']['goodsName'])
                datelst.append(reputation_data['reputation']['timeDesc'])
                sizelst.append(reputation_data['reputationProduct']['size'])
                collst.append(reputation_data['reputationProduct']['colorInfo'])
                satisfied.append(reputation_data['reputation']['satisfiedStatus'])

                # Use the same approach to create tag_content_and_name_list for each tagInfos
                tag_infos = reputation_data['reputation']['tagInfos']
                tag_content_and_name_list = [f"({tag_info['tagName']}){tag_info['tagContent']}" for tag_info in tag_infos]
                
                # Join the list of strings into a single string
                tag_content_and_name_str = ', '.join(tag_content_and_name_list)
                
                contents.append(tag_content_and_name_str)
                add_date.append(' ')
                add_content.append(' ')
            
            #태그를 사용하지 않고 작성
            else:
                prodName.append(reputation_data['reputationProduct']['goodsName'])
                datelst.append(reputation_data['reputation']['timeDesc'])
                sizelst.append(reputation_data['reputationProduct']['size'])
                collst.append(reputation_data['reputationProduct']['colorInfo'])
                satisfied.append(reputation_data['reputation']['satisfiedStatus'])
                contents.append(reputation_data['reputation']['content'])
                add_date.append(' ')
                add_content.append(' ')

In [102]:
df_temp = pd.DataFrame({
            'ProdName' : prodName,
            'Satisfied': satisfied,
            'Date': datelst,
            'Color': collst,
            'Size': sizelst,
            'Content': contents,
            'Additional_date':add_date,
            'Additional_review':add_content
        })

df_temp.to_excel('임시.xlsx')

## 상품 아이디 값/ 링크 가져오기

In [32]:
item_list = []
url = f'https://mapi.vip.com/vips-mobile/rest/shopping/pc/search/product/rank?callback=getMerchandiseIds&app_name=shop_pc&app_version=4.0&warehouse=VIP_NH&fdc_area_id=104104101&client=pc&mobile_platform=1&province_id=104104&api_key=70f71280d5d547b2a7bb370a529aeea1&user_id=&mars_cid=1705024509051_edef3b3ae16b6c6554148568bd9f9f84&wap_consumer=a&standby_id=nature&keyword=NEWBALANCE&lv3CatIds=&lv2CatIds=&lv1CatIds=&brandStoreSns=10012855&props=&priceMin=&priceMax=&vipService=&sort=6&pageOffset=120&channelId=1&gPlatform=PC&batchSize=120&_=1705995274631'
headers = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'Referer': 'https://category.vip.com/',
    'Sec-Ch-Ua': '"Whale";v="3", "Not-A.Brand";v="8", "Chromium";v="120"',
    'Sec-Ch-Ua-Mobile': '?0',
    'Sec-Ch-Ua-Platform': '"Windows"',
    'Sec-Fetch-Dest': 'script',
    'Sec-Fetch-Mode': 'no-cors',
    'Sec-Fetch-Site': 'same-site',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

response = requests.get(url, headers=headers)

try:
    # HTML을 bs으로 파싱
    soup = bs(response.text, 'html.parser')

    # JavaScript 함수 getCommentDataCb의 호출 부분을 정규 표현식으로 찾기
    pattern = re.compile(r'getMerchandiseIds\((.*?)\)', re.DOTALL)
    match = pattern.search(str(soup))

    if match:
        # 괄호 안의 JSON 형식의 데이터 추출
        comment_data_str = match.group(1)

        # JSON 형식의 데이터를 파싱하여 dict로 변환
        comment_data = json.loads(comment_data_str)
        comment_page_list.append(comment_data)
        
    else:
        print('No match found for getCommentDataCb')

except Exception as e:
    print(f'An unexpected error occurred: {e}')

n+=120




item_list = [item for item in comment_page_list if isinstance(item, dict) and 'data' in item and item['data']]
item_list

No match found for getCommentDataCb


[{'code': 1,
  'msg': 'success',
  'tid': '-5997131688633818039',
  'data': {'keepTime': 180,
   'total': 1042,
   'totalTxt': '',
   'batchSize': 120,
   'pageSize': 30,
   'isLast': 0,
   'showBsFilter': 1,
   'headInfo': {'isShowLabel': 1, 'listStyle': '2'},
   'products': [{'pid': '6920262500082066013'},
    {'pid': '6920133410056187279'},
    {'pid': '6919643519642876061'},
    {'pid': '6919939291521270237'},
    {'pid': '6920042067776276559'},
    {'pid': '6920042067742644303'},
    {'pid': '6920316079874014045'},
    {'pid': '6920698765685184399'},
    {'pid': '6920548033649420687'},
    {'pid': '6919939291521245661'},
    {'pid': '6920316079941385053'},
    {'pid': '6920679959959048527'},
    {'pid': '6920418511684687951'},
    {'pid': '6920524472405555037'},
    {'pid': '6920720382635969807'},
    {'pid': '6919833262836715869'},
    {'pid': '6920118761182129179'},
    {'pid': '6919946533225146063'},
    {'pid': '6920524472405546845'},
    {'pid': '6920421805126604675'},
    {'

In [15]:
item_list = []
n = 0 

for i in range(0,10):
    url = f'https://mapi.vip.com/vips-mobile/rest/shopping/pc/product/module/list/v2?callback=getMerchandiseDroplets2&app_name=shop_pc&app_version=4.0&warehouse=VIP_NH&fdc_area_id=104104101&client=pc&mobile_platform=1&province_id=104104&api_key=70f71280d5d547b2a7bb370a529aeea1&user_id=&mars_cid=1705024509051_edef3b3ae16b6c6554148568bd9f9f84&wap_consumer=a&productIds=6919643519642876061%2C6920589663969942619%2C6920617378574999747%2C6920546296198240861%2C6920737889675367389%2C6920127733187489167%2C6919839063213579165%2C6920546296198310493%2C6920546296198281821%2C6920546296198228573%2C6920616263223353539%2C6920617620210276125%2C6919946533241972431%2C6920616263206105283%2C6920184207467688591%2C6920590090768387343%2C6920616263223103683%2C6920421805126604675%2C6920604578256143567%2C6920513308907999451%2C6920377139601783183%2C6920607346487487643%2C6920616263223230659%2C6920622187294482627%2C6920483859831618959%2C6920088574257041475%2C6920123323190258139%2C6920647944778221583%2C6920677065873750363%2C6920269943379753539%2C6920604578256110799%2C6920159433752359247%2C6920606080150252699%2C6920029152250156175%2C6920563618330891293%2C6920616263223079107%2C6920617378575003843%2C6920563618330883101%2C6920026175085720987%2C6920441883216745615%2C6920620216329670851%2C6920617620192978717%2C6919939291521253853%2C6920559691502229583%2C6920036520498247323%2C6920679959959060815%2C6920036520582358683%2C6920546296198212189%2C6920620216294736067%2C6920729091783081179&scene=brandStore&standby_id=nature&extParams=%7B%22multiBrandStore%22%3A%220%22%2C%22stdSizeVids%22%3A%22%22%2C%22preheatTipsVer%22%3A%223%22%2C%22couponVer%22%3A%22v2%22%2C%22exclusivePrice%22%3A%221%22%2C%22iconSpec%22%3A%222x%22%2C%22ic2label%22%3A1%2C%22superHot%22%3A1%2C%22bigBrand%22%3A%221%22%7D&context=%7B%22615%22%3A%221%22%2C%22872%22%3A%221%22%7D&_=1706156807394'
    headers = {
        'Accept': '*/*',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
        'Referer': 'https://category.vip.com/',
        'Sec-Ch-Ua': '"Whale";v="3", "Not-A.Brand";v="8", "Chromium";v="120"',
        'Sec-Ch-Ua-Mobile': '?0',
        'Sec-Ch-Ua-Platform': '"Windows"',
        'Sec-Fetch-Dest': 'script',
        'Sec-Fetch-Mode': 'no-cors',
        'Sec-Fetch-Site': 'same-site',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }

    response = requests.get(url, headers=headers)

    try:
        # HTML을 bs으로 파싱
        soup = bs(response.text, 'html.parser')

        # JavaScript 함수 getCommentDataCb의 호출 부분을 정규 표현식으로 찾기
        pattern = re.compile(r'getMerchandiseIds\((.*?)\)', re.DOTALL)
        match = pattern.search(str(soup))

        if match:
            # 괄호 안의 JSON 형식의 데이터 추출
            comment_data_str = match.group(1)

            # JSON 형식의 데이터를 파싱하여 dict로 변환
            comment_data = json.loads(comment_data_str)
            
            if comment_data['data'] == []:
                break
            else:
                item_list.append(comment_data)
                print(item_list)
            
        else:
            print('No match found for getCommentDataCb')

    except Exception as e:
        print(f'An unexpected error occurred: {e}')

    n = (i+1)*120




items_list = [item for item in item_list if isinstance(item, dict) and 'data' in item and item['data']]

No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb
No match found for getCommentDataCb


In [13]:
items_list

[]

In [68]:
d = []
data_list = item_list[24]['data']['products']
pid_values = [d['pid'] for d in data_list]
modified_list = [item + '%' for item in pid_values]
modified_list

['6920647944778201103%',
 '6920604578256110799%',
 '6920432060725721629%',
 '6920616263206105283%',
 '6920607346487819419%',
 '6920036520430892699%',
 '6920011512412445391%',
 '6920128931130211727%',
 '6920421805143504771%',
 '6920029152351024271%',
 '6920616263240261827%',
 '6920482960889531037%',
 '6920589663969942619%',
 '6920620216329392323%',
 '6920639184266426843%',
 '6920607346487422107%',
 '6920622187277054147%',
 '6920635100358288847%',
 '6920218798226719759%',
 '6920671061312584027%',
 '6920559691502151759%',
 '6919833262836724061%',
 '6920139068079499611%',
 '6920616263223103683%',
 '6920629246819452701%',
 '6920622187277402307%',
 '6920589663969447003%',
 '6920604578256114895%',
 '6920616263223079107%',
 '6920620216311894211%',
 '6920513308873740507%',
 '6920671159969362255%',
 '6920610367397747163%',
 '6920456529700577693%',
 '6920604578256123087%',
 '6920295756370872067%',
 '6920617620210272029%',
 '6920620216329670851%',
 '6920622187277258947%',
 '6920589663969721435%',


전체 상품 크롤링

In [83]:
spuID_list = []


url = f'https://mapi.vip.com/vips-mobile/rest/shopping/pc/product/module/list/v2?callback=getMerchandiseDroplets2&app_name=shop_pc&app_version=4.0&warehouse=VIP_NH&fdc_area_id=104104101&client=pc&mobile_platform=1&province_id=104104&api_key=70f71280d5d547b2a7bb370a529aeea1&user_id=&mars_cid=1705024509051_edef3b3ae16b6c6554148568bd9f9f84&wap_consumer=a&productIds=6919643519642876061%2C6920589663969942619%2C6920617378574999747%2C6920546296198240861%2C6920737889675367389%2C6920127733187489167%2C6919839063213579165%2C6920546296198310493%2C6920546296198281821%2C6920546296198228573%2C6920616263223353539%2C6920617620210276125%2C6919946533241972431%2C6920616263206105283%2C6920184207467688591%2C6920590090768387343%2C6920616263223103683%2C6920421805126604675%2C6920604578256143567%2C6920513308907999451%2C6920377139601783183%2C6920607346487487643%2C6920616263223230659%2C6920622187294482627%2C6920483859831618959%2C6920088574257041475%2C6920123323190258139%2C6920647944778221583%2C6920677065873750363%2C6920269943379753539%2C6920604578256110799%2C6920159433752359247%2C6920606080150252699%2C6920029152250156175%2C6920563618330891293%2C6920616263223079107%2C6920617378575003843%2C6920563618330883101%2C6920026175085720987%2C6920441883216745615%2C6920620216329670851%2C6920617620192978717%2C6919939291521253853%2C6920559691502229583%2C6920036520498247323%2C6920679959959060815%2C6920036520582358683%2C6920546296198212189%2C6920620216294736067%2C6920729091783081179&scene=brandStore&standby_id=nature&extParams=%7B%22multiBrandStore%22%3A%220%22%2C%22stdSizeVids%22%3A%22%22%2C%22preheatTipsVer%22%3A%223%22%2C%22couponVer%22%3A%22v2%22%2C%22exclusivePrice%22%3A%221%22%2C%22iconSpec%22%3A%222x%22%2C%22ic2label%22%3A1%2C%22superHot%22%3A1%2C%22bigBrand%22%3A%221%22%7D&context=%7B%22615%22%3A%221%22%2C%22872%22%3A%221%22%7D&_=1706156807394'
headers = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'Cookie':'vip_address=%257B%2522pid%2522%253A%2522104104%2522%252C%2522cid%2522%253A%2522104104101%2522%252C%2522pname%2522%253A%2522%255Cu5e7f%255Cu4e1c%255Cu7701%2522%252C%2522cname%2522%253A%2522%255Cu5e7f%255Cu5dde%255Cu5e02%2522%257D; vip_province=104104; vip_province_name=%E5%B9%BF%E4%B8%9C%E7%9C%81; vip_city_name=%E5%B9%BF%E5%B7%9E%E5%B8%82; vip_city_code=104104101; vip_wh=VIP_NH; vip_ipver=31; mars_cid=1705024509051_edef3b3ae16b6c6554148568bd9f9f84; mars_sid=71cc1fefb049a7540897d1914e0fc634; mars_pid=3; vip_tracker_source_from=; user_class=a; VipUINFO=luc%3Aa%7Csuc%3Aa%7Cbct%3Ac_new%7Chct%3Ac_new%7Cbdts%3A0%7Cbcts%3A0%7Ckfts%3A0%7Cc10%3A0%7Crcabt%3A0%7Cp2%3A0%7Cp3%3A1%7Cp4%3A0%7Cp5%3A0%7Cul%3A3105; mst_area_code=104104; vpc_uinfo=fr1352%3A0%2Cfr674%3AD1%2Cfr766%3A0%2Cfr896%3A0%2Cfr392%3A310505%2Cfr398%3A0%2Cfr408%3A0%2Cfr251%3AA%2Cfr1195%3A0%2Cfr848%3A0%2Cfr1196%3A0%2Cfr902%3A0%2Cfr901%3A0%2Cfr980%3A0%2Cfr1570%3A0%2Cfr713%3A0%2Cfr1575%3A0%2Cfr1051%3A0%2Cfr259%3AS0-4%2Cfr0901%3A0%2Cfr884%3A0%2Cfr863%3A0%2Cfr1527%3A0%2Cfr344%3A0%2Cfr249%3AA1%2Cfr328%3A3105%2Cfr1544%3A0%2Cfr1543%3A0%2Cfr1521%3A0; VipDFT=7; visit_id=67E76C5C214E4C2E171D6A61FB50ABD3; vip_access_times=%7B%22list%22%3A1%2C%22detail%22%3A3%7D; pg_session_no=65',
    'Referer': 'https://category.vip.com/',
    'Sec-Ch-Ua': '"Whale";v="3", "Not-A.Brand";v="8", "Chromium";v="120"',
    'Sec-Ch-Ua-Mobile': '?0',
    'Sec-Ch-Ua-Platform': '"Windows"',
    'Sec-Fetch-Dest': 'script',
    'Sec-Fetch-Mode': 'no-cors',
    'Sec-Fetch-Site': 'same-site',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

response = requests.get(url, headers=headers)

try:
    # HTML을 BeautifulSoup으로 파싱
    soup = bs(response.text, 'html.parser')

    # JavaScript 함수 getCommentDataCb의 호출 부분을 정규 표현식으로 찾기
    pattern = re.compile(r'getMerchandiseDroplets2\((.*?)\)', re.DOTALL)
    match = pattern.search(str(soup))

    if match:
        # 괄호 안의 JSON 형식의 데이터 추출
        comment_data_str = match.group(1)

        # JSON 형식의 데이터를 파싱하여 dict로 변환
        comment_data = json.loads(comment_data_str)
        
        if comment_data['data'] == []:
            print('break')
        else:
            spuID_list.append(comment_data)
        
    else:
        print('No match found for getCommentDataCb')

except Exception as e:
    print(f'An unexpected error occurred: {e}')





spuIDs_list = [item for item in spuID_list if isinstance(item, dict) and 'data' in item and item['data']]

In [84]:
spuIDs_list[0]['data']['products']

[{'productId': '6919643519642876061',
  'brandId': '1711947357',
  'brandStoreSn': '10012855',
  'categoryId': '384336',
  'spuId': '2457353633879089152',
  'skuId': '2457353633879093248',
  'status': '0',
  'title': '中性鞋565系列运动鞋低帮耐磨复古鞋跑步鞋ML565CLG',
  'smallImage': 'http://h2.appsimg.com/a.appsimg.com/upload/merchandise/pdcvis/618174/2021/1223/196/94a9eed3-1829-4398-ae71-d3c777af938b_420_531.jpg',
  'squareImage': 'http://h2.appsimg.com/a.appsimg.com/upload/merchandise/pdcvis/618174/2021/1223/172/2ad29aaa-c6c8-4d51-8522-6ee13b332afe.jpg',
  'icon': 'http://h2.appsimg.com/b.appsimg.com/upload/momin/2024/01/12/5/1705048791742.png',
  'price': {'priceType': 'special',
   'priceLabelType': 'text',
   'priceLabel': '特卖价',
   'salePrice': '443',
   'salePriceSuff': '',
   'saleDiscount': '6.3折',
   'marketPrice': '699',
   'sellTipsType': 'low_price_60d',
   'sellTips': '60天历史低价',
   'shortSellTipsType': 'low_price_60d',
   'shortSellTips': '60天历史低价',
   'mixPriceLabel': '6.3折'},
  'attrs': 

In [85]:
pid_values = []
brand_values = []
for i in range(0,len(spuIDs_list[0]['data']['products'])):
    pid_values.append(spuIDs_list[0]['data']['products'][i]['spuId'])
    brand_values.append(spuIDs_list[0]['data']['products'][i]['brandId'])


len(pid_values)
len(brand_values)

50

50

In [61]:
links = ['https://mapi.vip.com/vips-mobile/rest/content/reputation/queryBySpuId_for_pc?callback=getCommentDataCb&app_name=shop_pc&app_version=4.0&warehouse=VIP_NH&fdc_area_id=104104101&client=pc&mobile_platform=1&province_id=104104&api_key=70f71280d5d547b2a7bb370a529aeea1&user_id=&mars_cid=1705024509051_edef3b3ae16b6c6554148568bd9f9f84&wap_consumer=a&spuId=4338169526903775237&brandId=1712085487&page=1&pageSize=10&functions=angle&timestamp=1706165263000&keyWordNlp=%E5%85%A8%E9%83%A8-%E6%8C%89%E9%BB%98%E8%AE%A4%E6%8E%92%E5%BA%8F&_=1706165169133',
          'https://mapi.vip.com/vips-mobile/rest/content/reputation/queryBySpuId_for_pc?callback=getCommentDataCb&app_name=shop_pc&app_version=4.0&warehouse=VIP_NH&fdc_area_id=104104101&client=pc&mobile_platform=1&province_id=104104&api_key=70f71280d5d547b2a7bb370a529aeea1&user_id=&mars_cid=1705024509051_edef3b3ae16b6c6554148568bd9f9f84&wap_consumer=a&spuId=5932725292711759879&brandId=1712085487&page=1&pageSize=10&functions=angle&timestamp=1706165228000&keyWordNlp=%E5%85%A8%E9%83%A8-%E6%8C%89%E9%BB%98%E8%AE%A4%E6%8E%92%E5%BA%8F&_=1706165170376',
          'https://mapi.vip.com/vips-mobile/rest/content/reputation/queryBySpuId_for_pc?callback=getCommentDataCb&app_name=shop_pc&app_version=4.0&warehouse=VIP_NH&fdc_area_id=104104101&client=pc&mobile_platform=1&province_id=104104&api_key=70f71280d5d547b2a7bb370a529aeea1&user_id=&mars_cid=1705024509051_edef3b3ae16b6c6554148568bd9f9f84&wap_consumer=a&spuId=3258994515834580992&brandId=1711947357&page=1&pageSize=10&functions=angle&timestamp=1706165184000&keyWordNlp=%E5%85%A8%E9%83%A8-%E6%8C%89%E9%BB%98%E8%AE%A4%E6%8E%92%E5%BA%8F&_=1706165171675',
          'https://mapi.vip.com/vips-mobile/rest/content/reputation/queryBySpuId_for_pc?callback=getCommentDataCb&app_name=shop_pc&app_version=4.0&warehouse=VIP_NH&fdc_area_id=104104101&client=pc&mobile_platform=1&province_id=104104&api_key=70f71280d5d547b2a7bb370a529aeea1&user_id=&mars_cid=1705024509051_edef3b3ae16b6c6554148568bd9f9f84&wap_consumer=a&spuId=3747635185036103683&brandId=1712085487&page=1&pageSize=10&functions=angle&timestamp=1706165312000&keyWordNlp=%E5%85%A8%E9%83%A8-%E6%8C%89%E9%BB%98%E8%AE%A4%E6%8E%92%E5%BA%8F&_=1706165167698',
          'https://mapi.vip.com/vips-mobile/rest/content/reputation/queryBySpuId_for_pc?callback=getCommentDataCb&app_name=shop_pc&app_version=4.0&warehouse=VIP_NH&fdc_area_id=104104101&client=pc&mobile_platform=1&province_id=104104&api_key=70f71280d5d547b2a7bb370a529aeea1&user_id=&mars_cid=1705024509051_edef3b3ae16b6c6554148568bd9f9f84&wap_consumer=a&spuId=1370015870685597696&brandId=1711947357&page=1&pageSize=10&functions=angle&timestamp=1706165327000&keyWordNlp=%E5%85%A8%E9%83%A8-%E6%8C%89%E9%BB%98%E8%AE%A4%E6%8E%92%E5%BA%8F&_=1706165157686',
          'https://mapi.vip.com/vips-mobile/rest/content/reputation/queryBySpuId_for_pc?callback=getCommentDataCb&app_name=shop_pc&app_version=4.0&warehouse=VIP_NH&fdc_area_id=104104101&client=pc&mobile_platform=1&province_id=104104&api_key=70f71280d5d547b2a7bb370a529aeea1&user_id=&mars_cid=1705024509051_edef3b3ae16b6c6554148568bd9f9f84&wap_consumer=a&spuId=2457353633879089152&brandId=1711947357&page=1&pageSize=10&functions=angle&timestamp=1706165362000&keyWordNlp=%E5%85%A8%E9%83%A8-%E6%8C%89%E9%BB%98%E8%AE%A4%E6%8E%92%E5%BA%8F&_=1706165353306',
          'https://mapi.vip.com/vips-mobile/rest/content/reputation/queryBySpuId_for_pc?callback=getCommentDataCb&app_name=shop_pc&app_version=4.0&warehouse=VIP_NH&fdc_area_id=104104101&client=pc&mobile_platform=1&province_id=104104&api_key=70f71280d5d547b2a7bb370a529aeea1&user_id=&mars_cid=1705024509051_edef3b3ae16b6c6554148568bd9f9f84&wap_consumer=a&spuId=4348865576018780163&brandId=1712085487&page=1&pageSize=10&functions=angle&timestamp=1706165380000&keyWordNlp=%E5%85%A8%E9%83%A8-%E6%8C%89%E9%BB%98%E8%AE%A4%E6%8E%92%E5%BA%8F&_=1706165356582',
          'https://mapi.vip.com/vips-mobile/rest/content/reputation/queryBySpuId_for_pc?callback=getCommentDataCb&app_name=shop_pc&app_version=4.0&warehouse=VIP_NH&fdc_area_id=104104101&client=pc&mobile_platform=1&province_id=104104&api_key=70f71280d5d547b2a7bb370a529aeea1&user_id=&mars_cid=1705024509051_edef3b3ae16b6c6554148568bd9f9f84&wap_consumer=a&spuId=3760301518028320768&brandId=1711947357&page=1&pageSize=10&functions=angle&timestamp=1706165400000&keyWordNlp=%E5%85%A8%E9%83%A8-%E6%8C%89%E9%BB%98%E8%AE%A4%E6%8E%92%E5%BA%8F&_=1706165357059',
          'https://mapi.vip.com/vips-mobile/rest/content/reputation/queryBySpuId_for_pc?callback=getCommentDataCb&app_name=shop_pc&app_version=4.0&warehouse=VIP_NH&fdc_area_id=104104101&client=pc&mobile_platform=1&province_id=104104&api_key=70f71280d5d547b2a7bb370a529aeea1&user_id=&mars_cid=1705024509051_edef3b3ae16b6c6554148568bd9f9f84&wap_consumer=a&spuId=4359280178484785157&brandId=1712085487&page=1&pageSize=10&functions=angle&timestamp=1706165414000&keyWordNlp=%E5%85%A8%E9%83%A8-%E6%8C%89%E9%BB%98%E8%AE%A4%E6%8E%92%E5%BA%8F&_=1706165357487',
          'https://mapi.vip.com/vips-mobile/rest/content/reputation/queryBySpuId_for_pc?callback=getCommentDataCb&app_name=shop_pc&app_version=4.0&warehouse=VIP_NH&fdc_area_id=104104101&client=pc&mobile_platform=1&province_id=104104&api_key=70f71280d5d547b2a7bb370a529aeea1&user_id=&mars_cid=1705024509051_edef3b3ae16b6c6554148568bd9f9f84&wap_consumer=a&spuId=2885195804353413135&brandId=1712085487&page=1&pageSize=10&functions=angle&timestamp=1706165430000&keyWordNlp=%E5%85%A8%E9%83%A8-%E6%8C%89%E9%BB%98%E8%AE%A4%E6%8E%92%E5%BA%8F&_=1706165358135']


In [68]:
# 추출된 정보를 저장할 리스트들
spu_ids = []
brand_ids = []
timestamps = []

# 각 링크에서 필요한 정보를 추출하여 리스트에 저장
for link in links:
    spu_id = link.split('spuId=')[1].split('&')[0]
    brand_id = link.split('brandId=')[1].split('&')[0]
    timestamp = link.split('&timestamp=')[1].split('&')[0]

    # 맨 마지막의 _= 부분은 필요없으므로 제거
    timestamp = timestamp.split('&_=')[0]

    # 각 정보를 해당하는 리스트에 추가
    spu_ids.append(spu_id)
    brand_ids.append(brand_id)
    timestamps.append(timestamp)

# 결과 출력
print("추출된 spuId 리스트:", spu_ids)
print("추출된 brandId 리스트:", brand_ids)
print("추출된 timestamp 리스트:", timestamps)

len(timestamps)


추출된 spuId 리스트: ['4338169526903775237', '5932725292711759879', '3258994515834580992', '3747635185036103683', '1370015870685597696', '2457353633879089152', '4348865576018780163', '3760301518028320768', '4359280178484785157', '2885195804353413135']
추출된 brandId 리스트: ['1712085487', '1712085487', '1711947357', '1712085487', '1711947357', '1711947357', '1712085487', '1711947357', '1712085487', '1712085487']
추출된 timestamp 리스트: ['1706165263000', '1706165228000', '1706165184000', '1706165312000', '1706165327000', '1706165362000', '1706165380000', '1706165400000', '1706165414000', '1706165430000']


10

In [87]:

comment_page_list = []


for i in range(0,len(spu_ids)):
    n=1
    while True:


        url = f'https://mapi.vip.com/vips-mobile/rest/content/reputation/queryBySpuId_for_pc?callback=getCommentDataCb&app_name=shop_pc&app_version=4.0&warehouse=VIP_NH&fdc_area_id=104104101&client=pc&mobile_platform=1&province_id=104104&api_key=70f71280d5d547b2a7bb370a529aeea1&user_id=&mars_cid=1705024509051_edef3b3ae16b6c6554148568bd9f9f84&wap_consumer=a&spuId={pid_values[i]}&brandId=1711947357&page={n}&pageSize=10&functions=angle&timestamp=1706165263000&keyWordNlp=%E5%85%A8%E9%83%A8-%E6%8C%89%E9%BB%98%E8%AE%A4%E6%8E%92%E5%BA%8F'
        headers = {
            'Accept': '*/*',
            'Accept-Encoding': 'gzip, deflate, br',
            'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
            'Referer': 'https://detail.vip.com/',
            'Sec-Ch-Ua': '"Whale";v="3", "Not-A.Brand";v="8", "Chromium";v="120"',
            'Sec-Ch-Ua-Mobile': '?0',
            'Sec-Ch-Ua-Platform': '"Windows"',
            'Sec-Fetch-Dest': 'script',
            'Sec-Fetch-Mode': 'no-cors',
            'Sec-Fetch-Site': 'same-site',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
        }

        response = requests.get(url, headers=headers)

        try:
            # HTML을 BeautifulSoup으로 파싱
            soup = bs(response.text, 'html.parser')

            # JavaScript 함수 getCommentDataCb의 호출 부분을 정규 표현식으로 찾기
            pattern = re.compile(r'getCommentDataCb\((.*?)\)', re.DOTALL)
            match = pattern.search(str(soup))

            if match:
                # 괄호 안의 JSON 형식의 데이터 추출
                comment_data_str = match.group(1)

                # JSON 형식의 데이터를 파싱하여 dict로 변환
                comment_data = json.loads(comment_data_str)
                
                if comment_data['data'] == []:
                    break
                else:
                    comment_page_list.append(comment_data)

            else:
                print('No match found for getCommentDataCb')

        except Exception as e:
            print(f'An unexpected error occurred: {e}')

        n += 1
    
    filtered_comment_page_list = [item for item in comment_page_list if isinstance(item, dict) and 'data' in item and item['data']]

In [72]:
filtered_comment_page_list

[{'code': 1,
  'data': [{'pointVal': 0,
    'reputation': {'address': '中国',
     'allowShareFlag': '1',
     'cacheIndex': 'cache_01_18_reputation_4338169526903775237_1712085487_全部-按默认排序_1_10_null',
     'content': '',
     'hasVideo': False,
     'imageCount': 1,
     'imageList': [{'imageId': 198143944,
       'url': 'http://h2.appsimg.com/g.vpimg1.com/upload/ugc/p/55/2023/09/14/183/b859fd4890c818b9df4f1.5975.jpg',
       'xyPosistion': ''}],
     'imageType': 0,
     'imgRelevance': '1',
     'impresses': '',
     'isAnonymous': 'YES',
     'isCommentConvert': False,
     'isEssence': 'NO',
     'isTop': 'NO',
     'isUseful': False,
     'nlpClass': 'P',
     'nlpScore': 70,
     'postTime': 1694667767000,
     'repType': '0',
     'reputationId': 556742352,
     'satisfiedStatus': 'VERY_SATISFIED',
     'showDetail': False,
     'sizeInfo': '尺码偏小',
     'tagInfos': [{'tagContent': '街头休闲',
       'tagId': 2861,
       'tagName': '版型风格',
       'tagType': '1'},
      {'tagContent': 

50개 수집

In [88]:
prodName = []
datelst = []
sizelst = []
collst = []
satisfied= []
contents = []
add_date = []
add_content = []



for i in range(0, len(filtered_comment_page_list)):
    for j in range(0, len(filtered_comment_page_list[i]['data'])):
        reputation_data = filtered_comment_page_list[i]['data'][j]

        #추가 후기를 작성했을 경우
        if 'additionalComments' in reputation_data:
            #태그 형태로 후기를 남긴 경우
            if 'tagInfos' in reputation_data['reputation']:
                prodName.append(reputation_data['reputationProduct']['goodsName'])
                datelst.append(reputation_data['reputation']['timeDesc'])
                sizelst.append(reputation_data['reputationProduct']['size'])
                collst.append(reputation_data['reputationProduct']['colorInfo'])
                satisfied.append(reputation_data['reputation']['satisfiedStatus'])

                # Use the same approach to create tag_content_and_name_list for each tagInfos
                tag_infos = reputation_data['reputation']['tagInfos']
                tag_content_and_name_list = [f"({tag_info['tagName']}){tag_info['tagContent']}" for tag_info in tag_infos]
                
                # Join the list of strings into a single string
                tag_content_and_name_str = ', '.join(tag_content_and_name_list)
                
                contents.append(tag_content_and_name_str)
                add_date.append(reputation_data['additionalComments']['postTimeDesc'])
                add_content.append(reputation_data['additionalComments']['content'])

            else : 
                prodName.append(reputation_data['reputationProduct']['goodsName'])
                datelst.append(reputation_data['reputation']['timeDesc'])
                sizelst.append(reputation_data['reputationProduct']['size'])
                collst.append(reputation_data['reputationProduct']['colorInfo'])
                satisfied.append(reputation_data['reputation']['satisfiedStatus'])
                contents.append(reputation_data['reputation']['content'])
                add_date.append(reputation_data['additionalComments']['postTimeDesc'])
                add_content.append(reputation_data['additionalComments']['content'])



        
        #추가 후기 작성 안함
        else:
            #태그형태로 작성
            if 'tagInfos' in reputation_data['reputation']:
                prodName.append(reputation_data['reputationProduct']['goodsName'])
                datelst.append(reputation_data['reputation']['timeDesc'])
                sizelst.append(reputation_data['reputationProduct']['size'])
                collst.append(reputation_data['reputationProduct']['colorInfo'])
                satisfied.append(reputation_data['reputation']['satisfiedStatus'])

                # Use the same approach to create tag_content_and_name_list for each tagInfos
                tag_infos = reputation_data['reputation']['tagInfos']
                tag_content_and_name_list = [f"({tag_info['tagName']}){tag_info['tagContent']}" for tag_info in tag_infos]
                
                # Join the list of strings into a single string
                tag_content_and_name_str = ', '.join(tag_content_and_name_list)
                
                contents.append(tag_content_and_name_str)
                add_date.append(' ')
                add_content.append(' ')
            
            #태그를 사용하지 않고 작성
            else:
                prodName.append(reputation_data['reputationProduct']['goodsName'])
                datelst.append(reputation_data['reputation']['timeDesc'])
                sizelst.append(reputation_data['reputationProduct']['size'])
                collst.append(reputation_data['reputationProduct']['colorInfo'])
                satisfied.append(reputation_data['reputation']['satisfiedStatus'])
                contents.append(reputation_data['reputation']['content'])
                add_date.append(' ')
                add_content.append(' ')

In [89]:
df_temp = pd.DataFrame({
            'ProdName' : prodName,
            'Satisfied': satisfied,
            'Date': datelst,
            'Color': collst,
            'Size': sizelst,
            'Content': contents,
            'Additional_date':add_date,
            'Additional_review':add_content
        })
df_temp.to_excel('D:/업무 파일/한중팀/중국 RAD/50data.xlsx')

In [90]:
df_temp

,ProdName,Satisfied,Date,Color,Size,Content,Additional_date,Additional_review
0,中性鞋565系列运动鞋低帮耐磨复古鞋跑步鞋ML565CLG,VERY_SATISFIED,2023-11-24,灰色,44,"(尺码大小)适中, (产品性价比)性价比很高, (版型风格)休闲舒适，型很好看，上脚效果很好...",,
1,中性鞋565系列运动鞋低帮耐磨复古鞋跑步鞋ML565CLG,VERY_SATISFIED,2022-11-12,灰色,38,"(产品性价比)高, (版型风格)休闲时尚, (尺码大小)平时37 这款买的38刚合适，鞋子真...",,
2,中性鞋565系列运动鞋低帮耐磨复古鞋跑步鞋ML565CLG,VERY_SATISFIED,2023-11-15,灰色,38,"(尺码大小)适中, (产品性价比)高, (版型风格)舒服 合脚 轻便",,
3,中性鞋565系列运动鞋低帮耐磨复古鞋跑步鞋ML565CLG,VERY_SATISFIED,2023-11-11,灰色,37.5,"(尺码大小)看评论说偏小就买大半码，刚刚好。, (产品性价比)好看，显脚小。, (版型风格)...",,
4,中性鞋565系列运动鞋低帮耐磨复古鞋跑步鞋ML565CLG,VERY_SATISFIED,2023-11-06,灰色,40,"(尺码大小)听了客服建议，非常合脚！, (产品性价比)物有所值！, (版型风格)简约舒适！",,
...,...,...,...,...,...,...,...,...
728,男装运动服休闲宽松训练连帽夹克外套MJ31536BK,VERY_SATISFIED,2023-11-16,黑色,2XL,质量好，上一件穿了很久，重新购入。,,
729,男装运动服休闲宽松训练连帽夹克外套MJ31536BK,VERY_SATISFIED,2024-01-07,黑色,2XL,太薄了一点，有点像防晒衣一样薄，一般般不太满意,,
730,女装运动服休闲宽松保暖立领夹克外套AWJ33320BK,VERY_SATISFIED,2023-11-17,黑色,L,"(服装风格)休闲运动套装, (搭配建议)搭配使用效果更佳, (上身效果)合适贴身衣物",,
731,女装运动服休闲宽松保暖立领夹克外套AWJ33320BK,VERY_SATISFIED,2024-01-20,黑色,XL,皮毛一体的、时尚休闲、喜欢值得拥有,,
